In [2]:
!pip install tqdm

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import codecs
from konlpy.tag import Okt
from konlpy.tag import Hannanum
from gensim.models.word2vec import Word2Vec
#import MaCab
import jpype
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import string
import warnings
from tqdm import tqdm 
from tqdm import trange
import time

warnings.filterwarnings("ignore", category=DeprecationWarning)

## naver temma stock data crawling

- x = page length  입력한 수 만큼 페이지를 불러온다

In [2]:
point = []
name = []
def temma_name(x):
    page_Num = x
    for i in tqdm(range(1,page_Num)):
        URL = 'https://finance.naver.com/sise/theme.nhn?'
        res = requests.get(URL + '&page={}'.format(str(i)))
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
        select_one = soup.select_one('td.col_type2').text
        select = soup.select('td.col_type2')
        for select_one in select:
            point.append(select_one.text.replace('\n\n\t\t\t\t','').replace('\n\t\t\t\t\n','').replace('\n',''))
    for i in range(1,page_Num):
        select_one = soup.select_one('td.col_type1').text
        select = soup.select('td.col_type1')
        for select_one in select:
            name.append(select_one.text)
    dic = { '종목명' : name,
            '득락률' : point}
    temma = pd.DataFrame(dic)
    return temma
temma_name(3)

100%|██████████| 2/2 [00:00<00:00,  2.18it/s]


,종목명,득락률
0,모바일솔루션(스마트폰),+10.37%
1,방위산업/전쟁 및 테러,+9.15%
2,케이블TV SO/MSO,+7.42%
3,아이폰,+4.04%
4,플렉서블 디스플레이,+3.60%
...,...,...
75,모바일게임(스마트폰),+0.65%
76,2차전지,+0.63%
77,2차전지(장비),+0.63%
78,핀테크(FinTech),+0.59%


- 현재 news, date, title 의 length가 맞지 않음.. 오류 찾는중

In [8]:
def news_crawling(comment):
    start = input('시작 날짜를 입력하시오. (e.g. : 2020.01.01) : ')
    end = input('종료 날짜를 입력하시오. (e.g. : 2020.12.31) : ')
    i = input('페이지 수를 입력하시오 : ')
    pageNum = 1
    lastpage = int(i) * 10 - 9
    news = []
    date = []
    title = []
    while pageNum < lastpage + 1:
        url = 'https://search.naver.com/search.naver?&where=news&query={}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={}&de={}&docid=&nso=so:r,p:from20210101to20210201,a:all&mynews=0&cluster_rank=14&start={}&refresh_start=0'.format(comment,start,end,pageNum)
        res = requests.get(url)
        html = res.text
        soup = BeautifulSoup(html,'html.parser')
    # 기사 출처 가져오기
        name_all = soup.find_all('a', class_ = 'info press')
        for name in name_all:
            news.append(name.text)
    #기사 작성일자 가져오기.. 오류발생
        #time_all = soup.find_all('span', class_ = 'info')
        #for time in time_all:
        #    date.append(time.text)
    #기사 제목 가져오기
        tit_all = soup.find_all('a', class_ = 'news_tit')
        for tit in tit_all:
            title.append(tit.text)
        pageNum += 10
    dic = {'신문사' : news,
#           '날짜' : date,
           '기사 제목' : title}
    headline = pd.DataFrame(dic)
    return headline
news_crawling('세종시 부동산')

시작 날짜를 입력하시오. (e.g. : 2020.01.01) : 2020.01.01
종료 날짜를 입력하시오. (e.g. : 2020.12.31) : 2020.01.01
페이지 수를 입력하시오 : 1


,신문사,기사 제목
0,머니투데이,[선택2020]총선 때마다 활개치는 '부동산 정치'
1,충청투데이,[충청민심 리포트] 대전역세권개발·세종시행정수도·충북선철도·충남일자리...
2,이데일리,[미리보는 이데일리신문]출산독려 대신 육아인프라 집중…세종시를 보라
3,스트레이트뉴스,"[2020년 전망] 부동산 숨고르고, 21대 총선 예측불허 , 정시 확대 중3 교육..."
4,대전MBC,[리포트]세종 싼 전세는 옛말!
5,국민일보,"7.52% 뛴 오피스텔 기준시가, 새해는 1.36% 오른다"
6,세계일보,“새해 경제 완만한 반등 예상… 기업투자 환경 조성 중요” [전문가 3인의 경...
7,충청일보,50∼299인 기업 週 52시간제 적용… 축산물이력제 닭·오리까지
8,글로벌이코노믹,새해 여는 1월 분양 '서울·수도권 청약열기' 전망...일반분양 73% 집중
9,글로벌이코노믹,연말 12월 주택 매매가격 0.38% 상승 “12.16대책 발표 전 조사값”


In [37]:
def title_text(comment):
    start = input('시작 날짜를 입력하시오. (e.g. : 2020.01.01) : ')
    end = input('종료 날짜를 입력하시오. (e.g. : 2020.12.31) : ')
    i = input('페이지 수를 입력하시오 : ')
    file_name = input('저장할 파일 이름을 작성하시오 : ')
    pageNum = 1
    page = 1
    lastpage = int(i) * 10 - 9
    title = []
    with open(str(file_name) + '.txt','a',encoding= 'utf-8') as f:
        while pageNum < lastpage + 1:
            url = 'https://search.naver.com/search.naver?&where=news&query={}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={}&de={}&docid=&nso=so:r,p:from20210101to20210201,a:all&mynews=0&cluster_rank=14&start={}&refresh_start=0'.format(comment,start,end,pageNum)
            res = requests.get(url)
            rescode = res.status_code
            if (rescode ==200) :
                html = res.text
                soup = BeautifulSoup(html,'html.parser')
                tit_all = soup.find_all('a', class_ = 'news_tit')
                for tit in tit_all:
                    title.append(tit.text)
                pageNum += 10
#                print(f'{page} 페이지 저장 완료')
#                page += 1
            else:
                print("Error Code:" + rescode)
        title = ' '.join(title)
        title = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9% ]',"",title)
        f.write(title)
        f.close
    return

In [38]:
title_text('세종시 부동산')

시작 날짜를 입력하시오. (e.g. : 2020.01.01) : 2020.01.01
종료 날짜를 입력하시오. (e.g. : 2020.12.31) : 2020.12.31
페이지 수를 입력하시오 : 5
저장할 파일 이름을 작성하시오 : test


In [2]:
title = pd.read_csv('news_title.txt')
title

""


### NLP 하는 과정 Dead Kernel 

In [1]:
# kkma = Kkma()
# komoran = Komoran()
# hannanum = Hannanum()
# mecab = MeCab.Tagger()
# #mecab = Mecab()
# okt = Okt()

NameError: name 'Kkma' is not defined